# UC Scholar - Final Product

### Yuanjia (Scott) Yang
#### Compiled at Aug 19th, 2022

This notebook is a batch report and documentation of the scripts I have developed over the summer during the UC Scholars Summer Research Program. This notebook contains summaries of the findings and some code snippets for demonstration purposes.

In [1]:
import scipy.io
import pandas as pd
import numpy as np 
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import ledoit_wolf
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, roc_auc_score
import scipy.stats as stats
from mat_preproc import preproc
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_style("darkgrid")

import matlab.engine
eng = matlab.engine.start_matlab()

# class attributes
source_info = ["SC", "CR", "SI", "M", "FA"]
response_info = ["RS", "RO", "F", "MN", "SN"]

# the x-axis on the projection graph
x_axis = [(1, 1), (3, 1), (5, 1), (1, 2), (5, 2),
          (1, 3), (3, 3), (5, 3), (4, 4), (2, 4), (4, 5), (2, 5)]


# Accuracy Report

## Objective
This replication merely focuses on the machine learning of the Linear Discriminant Analysis (LDA) section of the literature reported by Keuida et al. (2021) <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) while leaving the electroencephalogram (EEG) preprocessing untouched. In other words, we directly used the input features preprocessed and used by Kueida et al. and fed the data into LDA. Attempting to confirm the validity of the LDA between Kueida’s and sklearn’s implementation, we constructed the machine learning pipeline using sklearn to get relevant reports in the paper. 

## Evaluation Methods
Though unable to find the exact implementations and records of how Kueida calculated the reported values, we scrupulously followed the descriptions in the literature and replicated the result as best we could. First, leave-one-subject-out cross-validation was built. Within each fold, an LDA, with auto shrinkage along with an eigen solver, was fitted on the training data. Subsequently, the model's accuracy was evaluated based on the left-out subject. Since the literature mentioned that “The accuracy of the classifiers were calculated on balanced test data,” when using an imbalanced test-set (left-out subject) to evaluate the model’s performance within the fold, we randomly drop out the observations from the outweighed class (the class that has more observation from other) and calculate the accuracy based on the balanced test set. We repetitively balanced the test-set 10 times to smooth the randomness effect. Since we also want to evaluate the extremeness of our measurement, we repetitively execute the LOSO evaluation of the generalization error, as previously described, 10,000 times to generate the empirical distribution of the measurement. Consequently, p-values were calculated and contrasted with the reported values.

In [2]:
def cal_acc_balanced(clf, trail_num):
    """
    A summary function that calculate the accuracy in the literature
    based the aforementioned approach. 
    
    clf and trail_num can be used to specify which classifier and which
    experiment that we wish to learn.
    """
    file_path = f"data_imbalLDA_{trail_num}.mat"
    data_preproc = preproc(file_path, trail_num)
    if clf == "SN_MN":
        pos1, neg1 = data_preproc.filter_index(2,5,2,4)
        pos2, neg2 = data_preproc.filter_index(4,5,4,4)
    elif clf == "F_CR":
        pos1, neg1 = data_preproc.filter_index(1,3,2,4)
        pos2, neg2 = data_preproc.filter_index(3,3,2,5)
    else:
        raise ValueError("Unknown Classifier. Should be either `SN_MN` or `F_CR`")
    pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
    X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx)

    logo = LeaveOneGroupOut()

    scores = []

    for train_idx, test_idx in logo.split(X, y, subject):
        X_train, y_train = X[train_idx,:], y[train_idx]
        X_test, y_test = X[test_idx,:], y[test_idx]
        LDA = LinearDiscriminantAnalysis(shrinkage = None, solver = 'eigen')
        LDA.fit(X_train, y_train)
        # randomly drop datapoint to balance class
        pos_idx, neg_idx = np.arange(len(test_idx))[y_test == 1], np.arange(len(test_idx))[y_test != 1]
        pos_len, neg_len = len(pos_idx), len(neg_idx)
        acc = []
        for _ in range(10):
            if pos_len > neg_len:
                # when there are more positive class than negative
                # randomly drop positive class to equivalent the negative class
                pos_chosen = np.random.choice(pos_idx, neg_len, replace=False)
                neg_chosen = neg_idx
            else:
                pos_chosen = pos_idx
                neg_chosen = np.random.choice(neg_idx, pos_len, replace=False)
            filter_test_idx = np.concatenate([pos_chosen, neg_chosen])
            X_test_balanced, y_test_balanced = X_test[filter_test_idx, :], y_test[filter_test_idx]
            assert sum(y_test_balanced) == 0 # to check whether they are balanced class
            acc.append(LDA.score(X_test_balanced, y_test_balanced))
        acc = np.array(acc)
        scores.append(acc)
    scores = np.array(scores)
    return scores.mean()

np.random.seed(42)
accs = [
    [cal_acc_balanced("SN_MN", 1), cal_acc_balanced("F_CR", 1)],
    [cal_acc_balanced("SN_MN", 2),  cal_acc_balanced("F_CR", 2)]
]
accs

[[0.5304756678134714, 0.515777909215031],
 [0.5559914866368215, 0.5130120075747037]]

## Results
Our classifier got consistently lower accuracy scores in a single trial test set error evaluation of the accuracy compared with the reported value. 

![table 1](img/sim_vs_reported.png)

After simulating the evaluation of the classifier, all the reported values had a p-value that closed to 0 in our empirical distribution. We also explore several potential factors, including subjects with fewer than 10 observations in either positive or negative classes and using bootstrap instead of random drop-out. 

![MC](img/batch_MC.png)

For more details on how we ran the simulation, please refer to [AccuracySimulation-Analysis.ipynb](https://github.com/FleischerResearchLab/EEG-Familiarity-Prediction/blob/423f6a5649031436d3e9c052c98abbba778ffa2d/AccuracySimulation-Analysis.ipynb) and [AccuracySimulation.ipynb](https://github.com/FleischerResearchLab/EEG-Familiarity-Prediction/blob/423f6a5649031436d3e9c052c98abbba778ffa2d/AccuracySimulation.ipynb).

# Projection Report

## Objective
To further validate the credibility of our LDA model, we created the projection of each subject’s observation onto the discriminant learned by the previously trained classifier. We wish to see a similar projection trend of different classes in the binary classification settings. A similar projection trend across classes indicates a similar discriminant has been observed. 

## Methods
Previously Kueida projected all the participant’s observations onto the discriminant during the training process. We strictly followed the implementation of calculating the projection values within the LOSO processes. Specifically, within the LOSO, an LDA was trained based on the features excluding the chosen subject, and subsequently, a discriminant was learned. Then, all observations of the left-out subjects were projected onto the aforementioned discriminant. After the LOSO, every participant’s observations were projected onto the discriminant learned by all other participants’ observations. Subsequently, summary statistics were calculated based on the projection of different classes, regardless of the participant’s identity. Last but not least, a projection graph was plotted based on the summary statistics calculated.

For implementation details, please refer to [mat_preproc.py's `generate_projections()`](https://github.com/FleischerResearchLab/EEG-Familiarity-Prediction/blob/423f6a5649031436d3e9c052c98abbba778ffa2d/mat_preproc.py#L302)

## Results

After plotting SN v.s. MN and F v.s. CR projection in two experiments, we found similar but not identical projections.

![table1](img/table_projections.png)
<center>Table of Graph 1: Replication of Projection</center>

![table2](img/table_reported_projections.png)
<center>Table of Graph 2: Reported Projection</center>

Comparing the two tables of graphs, the overall trend of projection looks verisimilar yet not identical. Specifically, SN v.s. MN classifier’s projection in both experiments show a similar trend of means and width of confidence interval, except for the FA-RO classifier where we observed higher variance compared with reported projection. For F v.s. CR projection, we observed this interesting trend where it is generally concave in our replication project while convex in the reported projection. 

While the x-axis does not intrinsically have a chronological order, this reverse in trend might be a result of reversing the positive and negative class. In other words, we believed that, because our replicated projections resemble the reported results, two implementations of LDA are similar in finding the discriminant of the two desired classes.

# Direct Comparism of Model Performance - Trainning-Set Error

## Methods

We compared the training error on the entire dataset, turning off shrinkage in both Matlab and python codebases.  This was the most straightforward setup, with no possible differences in performance from anything other than LDA implementation. This section is mainly concerns with what will be the possible differences between the Linear Discriminant Analysis implemented by Sklearn and by Matlab.

In [8]:
def clf_summary(clf, exp):
    """
    Summary function for the performance of the sklearn packages
    """
    file_path = f"data_imbalLDA_{exp}.mat"
    data_preproc = preproc(file_path, exp)
    if clf == "SN_vs_MN":
        pos1, neg1 = data_preproc.filter_index(2,5,2,4)
        pos2, neg2 = data_preproc.filter_index(4,5,4,4)
    elif clf == "F_vs_CR":
        pos1, neg1 = data_preproc.filter_index(1,3,2,4)
        pos2, neg2 = data_preproc.filter_index(3,3,2,5)
    else:
        raise ValueError("Unknown Classifier")
    pos_idx, neg_idx = data_preproc.merge_two_class(pos1, neg1, pos2, neg2)
    X, y, subject = data_preproc.get_data_by_index(pos_idx, neg_idx, eliminate_trails = False)
    print(f"Summary for Clf: {clf}, Exp: {exp}")
    print(f"the shape of the training features is \n    {X.shape} \n")

    ## use shrinkage 0 to fit the whole data and test on the whole data
    LDA = LinearDiscriminantAnalysis(shrinkage = 0, solver = 'eigen')
    LDA.fit(X, y)
    acc = LDA.score(X, y)
    print(f"the accuracy for experiment {exp} {clf} is {acc:.3f}")
    return acc, LDA.coef_, LDA.intercept_

def compare_performance(clf, exp):
    """
    Compare classifier performance between classifier implemented
    in two different language.
    
    For more info about the mechanism of matlab implementation,
    please refer to LDA_simplest_test.m in the same directories.
    """
    assert clf in ["SN_vs_MN", "F_vs_CR"], "Invalid Classifier"
    assert exp in [1, 2], "Invalid Experiment Number"
    # Python & Sklearn
    print(f"Python & Sklearn: \n")
    acc_p, W_p, b_p = clf_summary(clf, exp)
    
    # Matlab & Kueida
    print(f"\n \nMatlab: \n")
    acc_m, W_m, b_m = eng.LDA_simplest_test(clf, exp, nargout=3)
    W_m = np.array(W_m)
    
    cos = cosine_similarity(W_m.reshape(1, -1), W_p[0].reshape(1, -1))[0][0]
    outperformed = "outperformed" if acc_p > acc_m else "underperform"

    print(
f"""
Summary:

    The python's sklearn {outperformed} the matlab implementation. 
    The cosine similarity between two discriminants is: {cos:.5f};
    The bias of the python's discriminant is:           {b_p[0]:.5f};
    The bias of the matlab's discriminant is:           {b_m:.5f};
    """)

## Results
Using the data from experiment 1 of Keuida et al. (2021), sklearn’s training set accuracy outperformed the Matlab implementation (sklearn $acc=0.616$, Matlab $acc=0.612$). Furthermore, the learned discriminants between the two have high cosine similarity (1) while significantly varied in the bias terms by two orders of magnitude. In other words, we believe there are systematic differences between the LDA implemented in Matlab and sklearn, especially the bias calculation. Because of the discrepancy between the learned discriminants without shrinkage, we did not investigate if Matlab and sklearn implement shrinkage identically. It is worth noting that we explored different LDA solvers in Sklearn, which did not affect the model’s fit on this dataset. Hence, the differences between sklearn and Matlab are probably not due to solvers. 

We observed the same trend of similar weights, except for bias term, learned by Sklearn and  Matlab across classifiers and two experiments. In all trained classifiers, python and sklearn persistently outperform their Matlab counterpart. We suspect that the discrepancies in classifier performance are due to the bias term learned in Matlab.

### SN vs MN, Exp 1

In [9]:
compare_performance("SN_vs_MN", 1)

Python & Sklearn: 

Summary for Clf: SN_vs_MN, Exp: 1
the shape of the training features is 
    (3898, 72) 

the accuracy for experiment 1 SN_vs_MN is 0.616

 
Matlab: 

The shape of the training set is
        3898          72

k_reg Change to 0, to turn-off the shrinkage
The bias is:
    0.2947

the accuracy for experiment 1 SN_vs_MN is: 0.61724

Summary:

    The python's sklearn underperform the matlab implementation. 
    The cosine similarity between two discriminants is: 1.00000;
    The bias of the python's discriminant is:           0.29475;
    The bias of the matlab's discriminant is:           0.29475;
    


### SN vs MN, Exp 2

In [5]:
compare_performance("SN_vs_MN", 2)

Python & Sklearn: 

Summary for Clf: SN_vs_MN, Exp: 2
the shape of the training features is 
    (3133, 72) 

the accuracy for experiment 2 SN_vs_MN is 0.688

 
Matlab: 

The shape of the training set is
        3133          72

k_reg Change to 0, to turn-off the shrinkage
The bias is:
    0.7061

the accuracy for experiment 2 SN_vs_MN is: 0.6856

Summary:

    The python's sklearn outperformed the matlab implementation. 
    The cosine similarity between two discriminants is: 1.000;
    The bias of the python's discriminant is:           0.706;
    The bias of the matlab's discriminant is:           0.706;
    


### F vs CR, Exp 1

In [6]:
compare_performance("F_vs_CR", 1)

Python & Sklearn: 

Summary for Clf: F_vs_CR, Exp: 1
the shape of the training features is 
    (3703, 72) 

the accuracy for experiment 1 F_vs_CR is 0.705

 
Matlab: 

The shape of the training set is
        3703          72

k_reg Change to 0, to turn-off the shrinkage
The bias is:
   -0.9773

the accuracy for experiment 1 F_vs_CR is: 0.70672

Summary:

    The python's sklearn underperform the matlab implementation. 
    The cosine similarity between two discriminants is: 1.000;
    The bias of the python's discriminant is:           -0.977;
    The bias of the matlab's discriminant is:           -0.977;
    


### F vs CR, Exp 2

In [7]:
compare_performance("F_vs_CR", 2)

Python & Sklearn: 

Summary for Clf: F_vs_CR, Exp: 2
the shape of the training features is 
    (3326, 72) 

the accuracy for experiment 2 F_vs_CR is 0.738

 
Matlab: 

The shape of the training set is
        3326          72

k_reg Change to 0, to turn-off the shrinkage
The bias is:
   -1.0187

the accuracy for experiment 2 F_vs_CR is: 0.73722

Summary:

    The python's sklearn outperformed the matlab implementation. 
    The cosine similarity between two discriminants is: 1.000;
    The bias of the python's discriminant is:           -1.019;
    The bias of the matlab's discriminant is:           -1.019;
    


## Further Exploration to the Codebase of Bias Calculations

According to the equation 8.14 from Modern Multivariate Statistical Techniques<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2) indicate how to calculate the bias term of the learned discriminant. Specifically, in binary classification settings given the class means $\mu_1$ and $\mu_2$ and the covariance matrix $\Sigma_{XX}$, the intercept can be calculated by:

$$
b_0 = -\frac12 \left\{ \mu_1^T \Sigma_{XX}^{-1} \mu_1 - \mu_2^T \Sigma_{XX}^{-1} \mu_2 \right\} + \log_e\left( \frac{\pi_2}{\pi_1}\right)
$$


I trace down to sklearn’s codebase ([scikit-learn/sklearn/discriminant_analysis.py, at line 387-389](https://github.com/scikit-learn/scikit-learn/blob/36958fb240fbe435673a9e3c52e769f01f36bec0/sklearn/discriminant_analysis.py#L387)) and confirm that scikit does the same thing with the following code:

```python
self.intercept_ = -0.5 * np.diag(np.dot(self.means_, self.coef_.T)) + np.log(
    self.priors_
)
```

However, at file `/lkueida/Documents/MATLAB/MemoryFinale/LOSO/Exp1/check_lda_train_reg_auto.m` line 84, he calculated the B (bias) by the following formula:

```matlab
B = (m0'*W+m1'*W)/2;
```
where `W` is the learned discriminant and `m0`  and `m1` are the class means.

Up until this point, we have a clear understandings of how the the differences in the bias term might results in the performance differences. The next step, which we have not yet reach, is to validate whether there are still systematic differences between Leave-One-Subject-Out Cross Validation (LOSOCV)

# Discussion & Future Results:

Even though we have not fully replicated and validated the functioning toolset, the next exciting step is applying the machine learning pipeline to the independent dataset gathered by Addante et al. (2021) <a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3), which implemented a comparable experimental paradigm with Kueida et al. We have already contacted Addante et al. regarding the further exploration of the dataset. We are planning to understand how the dataset is constructed and in what ways it can be analyzed. Specifically, each observation in the dataset needs to be formatted and classified in a way that follows the same features as Kueida et al. preprocess their data. Given the formatted features and data inputs, we will be able to explore and train a spectrum of interpretable classifiers of different behaviors. Subsequently, we will evaluate each classifier’s performance and interpret the learned parameters. We hope the parameters of the trained machine learning models will shed light on the underlying processes of memory retrieval tasks.

To break the linear constraints of Linear Discriminant Analysis (LDA), we will explore several other classifiers, such as Quadratic Discriminant Analysis (QDA) and support vector machines (SVM) with/without kernel trick and logistic classification. We will also implement model selection to compare different models’ performance and choose the best one. 


# Citation

<a name="cite_note-1"></a>1. [^](#cite_ref-1) Liao, K., Mollison, M. V., Curran, T., & de Sa, V. R. (2021, January). EEG Reveals Familiarity by Controlling Confidence in Memory Retrieval. In Proceedings of the Annual Meeting of the Cognitive Science Society (Vol. 43).

<a name="cite_note-2"></a>2. [^](#cite_ref-2) Izenman, A. J. (2008). Modern multivariate statistical techniques. Regression, classification and manifold learning, 10, 978-0.

<a name="cite_note-3"></a>3. [^](#cite_ref-3) Muller, A., Sirianni, L. A., & Addante, R. J. (2021). Neural correlates of the Dunning–Kruger effect. European Journal of Neuroscience, 53(2), 460-484.

# Acknowledgement

- I wish to extend my special thanks to UC Scholars and Undergraduate Research Hub (URH) for providing scholarships for this project
- Professor Jason G. Fleischer offered invaluable guidance and helped with this project.
- Special thanks to Kueida and Virginia de Sa for providing valuable data and statistics for this project
- I would also like to thank my labmates Ruoxuan Li and Sahithi Chimuula for laying down the foundational work and assisting this project. 